We need to compile llama-cpp-python[server] using Nvidia Compile (nvcc). Make sure you are running a CUDA Notebook !

In [1]:
!NVCC_APPEND_FLAGS='-allow-unsupported-compiler' CUDACXX=/usr/local/cuda/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python[server] --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 214.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 242.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 270.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 238.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 224.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 289.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 245.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 259.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 286.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 208.2 MB/s eta 0:

Download a llama-3 model. This has a few quantized, finetuned
versions - https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF

In [2]:
!wget -O Meta-Llama-3-8B-Instruct-Q8_0.gguf https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct-Q8_0.gguf?download=true

--2024-04-30 04:04:40--  https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct-Q8_0.gguf?download=true
Resolving huggingface.co (huggingface.co)... 3.160.5.25, 3.160.5.76, 3.160.5.102, ...
Connecting to huggingface.co (huggingface.co)|3.160.5.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/26/a8/26a8d953d9f1bd768f71bbb8a45bf7b3383a6131813ea16bea728507393f4e5e/583c616da14b82930f887f991ab446711da0b029166200b67892d7c9f8f45958?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Meta-Llama-3-8B-Instruct-Q8_0.gguf%3B+filename%3D%22Meta-Llama-3-8B-Instruct-Q8_0.gguf%22%3B&Expires=1714709080&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDcwOTA4MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzI2L2E4LzI2YThkOTUzZDlmMWJkNzY4ZjcxYmJiOGE0NWJmN2IzMzgzYTYxMzE4MTNlYTE2YmVhNzI4NTA3Mzkz

Load the model making sure to specify to use the GPU with n_gpu_layers.

In [3]:
from llama_cpp import Llama
llm = Llama(model_path="Meta-Llama-3-8B-Instruct-Q8_0.gguf", n_gpu_layers=-1)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from Meta-Llama-3-8B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.he

This is where we specify the completions.

In [4]:
output = llm(
  "<PROMPT>", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)


llama_print_timings:        load time =     185.42 ms
llama_print_timings:      sample time =      63.24 ms /   118 runs   (    0.54 ms per token,  1865.85 tokens per second)
llama_print_timings: prompt eval time =     185.37 ms /     5 tokens (   37.07 ms per token,    26.97 tokens per second)
llama_print_timings:        eval time =    4120.03 ms /   117 runs   (   35.21 ms per token,    28.40 tokens per second)
llama_print_timings:       total time =    5310.63 ms /   122 tokens


In [5]:
llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant with an Australian style."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ]
)


llama_print_timings:        load time =     185.42 ms
llama_print_timings:      sample time =     256.66 ms /   473 runs   (    0.54 ms per token,  1842.89 tokens per second)
llama_print_timings: prompt eval time =     137.96 ms /    39 tokens (    3.54 ms per token,   282.68 tokens per second)
llama_print_timings:        eval time =   16823.70 ms /   472 runs   (   35.64 ms per token,    28.06 tokens per second)
llama_print_timings:       total time =   20818.97 ms /   511 tokens


{'id': 'chatcmpl-23e6feb9-19c0-43d6-aacd-6ff0c3996c2c',
 'object': 'chat.completion',
 'created': 1714449932,
 'model': 'Meta-Llama-3-8B-Instruct-Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'G\'day mate! Let me spin you a yarn \'bout the most ripper of creatures – llamas!\n\nIn the heart o\' the Andes, where the air\'s crisp and the grass is green, there lived a mob o\' llamas like no other. They were as tough as nails, with their soft fur and gentle eyes, but don\'t you worry, they had a bit o\' sass in \'em too.\n\nThere was Lola, the leader o\' the pack – a feisty little number with a coat as white as snow and a spirit to match. She\'d been around the block a few times, knowin\' all the best spots to find the tastiest grass and the coolest shade trees.\n\nThen there was her mate, Max – a big fella with a shaggy brown coat and a heart o\' gold. He was as gentle as a lamb (no pun intended), but don\'t mess with him or he\'d show you his spi

In [6]:
!python3 -m llama_cpp.server --model Meta-Llama-3-8B-Instruct-Q8_0.gguf --n_gpu_layers=-1 --host=0.0.0.0 --port=8080

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from Meta-Llama-3-8B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.he

Try browsing to the Route and Proxy that your notebook exposes. Be sure to create the two Routes in OpenShift first.

```yaml
oc apply -n rhods-notebooks -f - <<EOF
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: llama-openapi-json
  annotations:
    haproxy.router.openshift.io/rewrite-target: /notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/openapi.json
spec:
  host: jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com
  path: /openapi.json
  port:
    targetPort: oauth-proxy
  tls:
    termination: Reencrypt
    insecureEdgeTerminationPolicy: None
  to:
    kind: Service
    name: jupyter-nb-admin-tls
    weight: 100
  wildcardPolicy: None
EOF

oc apply -n rhods-notebooks -f - <<EOF
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: llama-v1
  annotations:
    haproxy.router.openshift.io/rewrite-target: /notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/v1
spec:
  host: jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com
  path: /v1
  port:
    targetPort: oauth-proxy
  tls:
    termination: Reencrypt
    insecureEdgeTerminationPolicy: None
  to:
    kind: Service
    name: jupyter-nb-admin-tls
    weight: 100
  wildcardPolicy: None
EOF
```


Open in a browser tab and try it out using swagger.

https://jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com/notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/docs